In [34]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 40)

from vivarium_research_prl.noise import corruption, fake_names, noisify
from vivarium_research_prl.find_kids import datasets, noisify_data


!date
!whoami
!uname -a
!pwd

Fri 03 Feb 2023 09:52:57 PM PST
ndbs
Linux int-slurm-sarchive-p0004 5.4.0-89-generic #100-Ubuntu SMP Fri Sep 24 14:50:10 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/code/ndbs/vivarium_research_prl/noise


In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
project_output_dir = '/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop'
output_subdir = 'results/vv_post_processing_first_middle_names/united_states_of_america/2023_02_02_10_16_21/'
output_subsubdir = 'final_results/2023_02_02_11_10_52'
output_dir = f'{project_output_dir}/{output_subdir}/{output_subsubdir}'

!ls -l $output_dir


total 846760
-rw-r--r-- 1 albrja IHME-Simulationscience 116145705 Feb  2 11:20 decennial_census_observer.csv.bz2
-rw-r--r-- 1 albrja IHME-Simulationscience   1436741 Feb  2 11:20 household_survey_observer_acs.csv.bz2
-rw-r--r-- 1 albrja IHME-Simulationscience   7443547 Feb  2 11:20 household_survey_observer_cps.csv.bz2
-rw-r--r-- 1 albrja IHME-Simulationscience  30294533 Feb  2 11:21 social_security_observer.csv.bz2
-rw-r--r-- 1 albrja IHME-Simulationscience 696671121 Feb  2 11:27 tax_w2_observer.csv.bz2
-rw-r--r-- 1 albrja IHME-Simulationscience  15056759 Feb  2 11:20 wic_observer.csv.bz2


In [8]:
%%time
df_wic = pd.read_csv(f'{output_dir}/wic_observer.csv.bz2')
df_wic

CPU times: user 2.58 s, sys: 195 ms, total: 2.77 s
Wall time: 2.77 s


,simulant_id,last_name_id,age,sex,race_ethnicity,date_of_birth,address_id,guardian_1,guardian_2,guardian_1_address_id,guardian_2_address_id,wic_year,random_seed,year_of_birth,first_name,middle_initial
0,7359_65,65.0,8.553950,Female,Black,2012-05-21,35,-1,-1,NaN,NaN,2021,7359,2012,Emma,M
1,7359_116,116.0,25.505874,Female,Black,1995-06-08,53,-1,-1,NaN,NaN,2021,7359,1995,Isabella,A
2,7359_212,211.0,30.291655,Female,Latino,1990-08-25,88,-1,-1,NaN,NaN,2021,7359,1990,Paige,L
3,7359_248,248.0,23.891573,Female,Black,1997-01-18,97912,-1,-1,NaN,NaN,2021,7359,1997,Michelle,M
4,7359_267,266.0,37.072670,Female,White,1983-11-13,109,-1,-1,NaN,NaN,2021,7359,1983,Maria,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593011,5020_277602,231367.0,0.041138,Male,Asian,2030-01-08,152642,231367,-1,152642.0,NaN,2030,5020,2030,Grayson,B
593012,5020_277607,237489.0,0.045699,Male,Latino,2030-01-07,135561,237490,-1,135561.0,NaN,2030,5020,2030,Tanner,E
593013,5020_277612,242914.0,0.039537,Female,Asian,2030-01-09,167694,242914,-1,167694.0,NaN,2030,5020,2030,Madison,S
593014,5020_277617,5210.0,0.054807,Male,Latino,2030-01-03,97484,5214,-1,97484.0,NaN,2030,5020,2030,Gavin,J


# Copy data to test noise functions

In [48]:
df_wic_noisy = df_wic.copy()

In [49]:
rng = np.random.default_rng(23939397502020575)
rng

Generator(PCG64) at 0x7FECC9F5E900

# Test code for handling missingness in `apply_noise_function`

In [50]:
df_wic.first_name.notna()

0         True
1         True
2         True
3         True
4         True
          ... 
593011    True
593012    True
593013    True
593014    True
593015    True
Name: first_name, Length: 593016, dtype: bool

## Replace some values of `first_name` with missing for testing

In [51]:
%%time
noisify.apply_noise_function_to_column(
        df_wic_noisy, 'first_name', 0.40, rng,
        corruption.replace_with_missing, share_random_state=False, inplace=True
    )
df_wic_noisy

CPU times: user 45.9 ms, sys: 7.48 ms, total: 53.4 ms
Wall time: 52.5 ms


,simulant_id,last_name_id,age,sex,race_ethnicity,date_of_birth,address_id,guardian_1,guardian_2,guardian_1_address_id,guardian_2_address_id,wic_year,random_seed,year_of_birth,first_name,middle_initial
0,7359_65,65.0,8.553950,Female,Black,2012-05-21,35,-1,-1,NaN,NaN,2021,7359,2012,NaN,M
1,7359_116,116.0,25.505874,Female,Black,1995-06-08,53,-1,-1,NaN,NaN,2021,7359,1995,NaN,A
2,7359_212,211.0,30.291655,Female,Latino,1990-08-25,88,-1,-1,NaN,NaN,2021,7359,1990,NaN,L
3,7359_248,248.0,23.891573,Female,Black,1997-01-18,97912,-1,-1,NaN,NaN,2021,7359,1997,NaN,M
4,7359_267,266.0,37.072670,Female,White,1983-11-13,109,-1,-1,NaN,NaN,2021,7359,1983,Maria,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593011,5020_277602,231367.0,0.041138,Male,Asian,2030-01-08,152642,231367,-1,152642.0,NaN,2030,5020,2030,Grayson,B
593012,5020_277607,237489.0,0.045699,Male,Latino,2030-01-07,135561,237490,-1,135561.0,NaN,2030,5020,2030,NaN,E
593013,5020_277612,242914.0,0.039537,Female,Asian,2030-01-09,167694,242914,-1,167694.0,NaN,2030,5020,2030,Madison,S
593014,5020_277617,5210.0,0.054807,Male,Latino,2030-01-03,97484,5214,-1,97484.0,NaN,2030,5020,2030,Gavin,J


In [52]:
corrupted = df_wic_noisy['first_name'].notna()
corrupted

0         False
1         False
2         False
3         False
4          True
          ...  
593011     True
593012    False
593013     True
593014     True
593015    False
Name: first_name, Length: 593016, dtype: bool

In [53]:
corrupted.loc[corrupted]

4         True
5         True
7         True
8         True
11        True
          ... 
593009    True
593010    True
593011    True
593013    True
593014    True
Name: first_name, Length: 354853, dtype: bool

In [54]:
corrupted.sum()

354853

In [55]:
df_wic_noisy

,simulant_id,last_name_id,age,sex,race_ethnicity,date_of_birth,address_id,guardian_1,guardian_2,guardian_1_address_id,guardian_2_address_id,wic_year,random_seed,year_of_birth,first_name,middle_initial
0,7359_65,65.0,8.553950,Female,Black,2012-05-21,35,-1,-1,NaN,NaN,2021,7359,2012,NaN,M
1,7359_116,116.0,25.505874,Female,Black,1995-06-08,53,-1,-1,NaN,NaN,2021,7359,1995,NaN,A
2,7359_212,211.0,30.291655,Female,Latino,1990-08-25,88,-1,-1,NaN,NaN,2021,7359,1990,NaN,L
3,7359_248,248.0,23.891573,Female,Black,1997-01-18,97912,-1,-1,NaN,NaN,2021,7359,1997,NaN,M
4,7359_267,266.0,37.072670,Female,White,1983-11-13,109,-1,-1,NaN,NaN,2021,7359,1983,Maria,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593011,5020_277602,231367.0,0.041138,Male,Asian,2030-01-08,152642,231367,-1,152642.0,NaN,2030,5020,2030,Grayson,B
593012,5020_277607,237489.0,0.045699,Male,Latino,2030-01-07,135561,237490,-1,135561.0,NaN,2030,5020,2030,NaN,E
593013,5020_277612,242914.0,0.039537,Female,Asian,2030-01-09,167694,242914,-1,167694.0,NaN,2030,5020,2030,Madison,S
593014,5020_277617,5210.0,0.054807,Male,Latino,2030-01-03,97484,5214,-1,97484.0,NaN,2030,5020,2030,Gavin,J


## Test updated `apply_noise_function_to_column` function with new ability to handle missing data

In [56]:
%%time
noisify.apply_noise_function_to_column(
        df_wic_noisy, 'first_name', 0.10, rng,
        corruption.ocr_corrupt, (1/3,), vectorized=False, inplace=True)

CPU times: user 810 ms, sys: 39.7 ms, total: 849 ms
Wall time: 796 ms


In [57]:
df_wic_noisy['first_name']

0             NaN
1             NaN
2             NaN
3             NaN
4           Maria
           ...   
593011    Grayson
593012        NaN
593013    Madison
593014      Gavin
593015        NaN
Name: first_name, Length: 593016, dtype: object

In [58]:
df_wic_noisy.loc[corrupted, 'first_name']

4            Maria
5            Jamie
7         Margaret
8            Emily
11         Diamond
            ...   
593009    Braelynn
593010     Paislee
593011     Grayson
593013     Madison
593014       Gavin
Name: first_name, Length: 354853, dtype: object

In [67]:
different = df_wic_noisy.loc[corrupted, 'first_name'].compare(
    df_wic.loc[corrupted, 'first_name']
)
different

,self,other
32,Katnna,Katrina
74,Jasnine,Jasmine
77,4mber,Amber
179,K:mber|y,Kimberly
198,Hailev,Hailey
...,...,...
592848,Amirab,Amirah
592849,CbIoe,Chloe
592894,Karelv,Karely
592899,Benjam:n,Benjamin


## Check that error rates are as expected

In [60]:
# Fraction of rows that are non-null (expected fraction 0.6)
corrupted.sum()/len(df_wic)

0.5983868900670472

In [68]:
# Actual fraction of rows that are different
len(different)/len(df_wic)

0.03341056565084247

In [69]:
# Actual fraction of different, non-null rows
len(different)/corrupted.sum()

0.055834387760565644

In [63]:
# Expected approximate fraction of different, non-null rows
.1*(1-.4)

0.06

In [64]:
# Expected approximate fraction of total rows that are different
.1*(1-.4)/2

0.03

# Do experiments to figure out how to generalize zipcode weights

In [75]:
p = np.array([0.04, 0.2, 0.36])
p/0.04

array([1., 5., 9.])

In [74]:
w = np.array([1., 5., 9.])
w/w.sum()

array([0.06666667, 0.33333333, 0.6       ])

In [76]:
p.sum()

0.6

In [77]:
p.mean()

0.19999999999999998

In [80]:
w.mean()

5.0

In [78]:
w/w.mean()

array([0.2, 1. , 1.8])

In [79]:
0.2*w/w.mean() # This is what we want

array([0.04, 0.2 , 0.36])

In [83]:
def zipcode_probs(mean_prob, w1, w2, w3):
    w = np.array([w1, w2, w3])
    return mean_prob * w / w.mean()

zipcode_probs(0.2, 0.04, 0.2, 0.36)

array([0.04, 0.2 , 0.36])

In [84]:
zipcode_probs(0.2, 1, 5, 9)

array([0.04, 0.2 , 0.36])

In [85]:
zipcode_probs(0.2, 1, 4, 10)

array([0.04, 0.16, 0.4 ])

# Test updated census noise

In [88]:
%%time
df_census = pd.read_csv(f'{output_dir}/decennial_census_observer.csv.bz2')
df_census

CPU times: user 28.7 s, sys: 471 ms, total: 29.2 s
Wall time: 29.4 s


,simulant_id,last_name_id,age,sex,race_ethnicity,date_of_birth,address_id,guardian_1,guardian_2,guardian_1_address_id,guardian_2_address_id,relation_to_household_head,census_year,housing_type,random_seed,year_of_birth,first_name,middle_initial
0,3254_0,0.0,67.376229,Female,Black,1952-11-15,6,-1,-1,NaN,NaN,Reference person,2020,Standard,3254,1952,Deanna,S
1,3254_1,0.0,37.501003,Male,Black,1982-10-01,6,-1,-1,NaN,NaN,Biological child,2020,Standard,3254,1982,Clifton,S
2,3254_2,0.0,31.157344,Male,Black,1989-02-03,6,-1,-1,NaN,NaN,Biological child,2020,Standard,3254,1989,Christopher,S
3,3254_3,3.0,37.944355,Male,Latino,1982-04-22,7,-1,-1,NaN,NaN,Reference person,2020,Standard,3254,1982,Travis,J
4,3254_4,4.0,60.986418,Male,White,1959-04-07,8,-1,-1,NaN,NaN,Reference person,2020,Standard,3254,1959,Davis,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4943280,3541_278165,44253.0,0.055643,Female,Latino,2030-03-28,150569,44253,-1,150569.0,NaN,Biological child,2030,Standard,3541,2030,Mila,C
4943281,3541_278166,50096.0,0.039506,Female,Black,2030-04-03,3,50097,-1,3.0,NaN,Noninstitutionalized GQ pop,2030,College,3541,2030,Aicha,L
4943282,3541_278167,113377.0,0.052213,Female,Black,2030-03-29,96003,113380,-1,96003.0,NaN,Biological child,2030,Standard,3541,2030,Amila,E
4943283,3541_278168,140942.0,0.016033,Female,Latino,2030-04-12,179963,140944,-1,179963.0,NaN,Grandchild,2030,Standard,3541,2030,Tallulah,S


In [94]:
%%time
def prep_census_data(df_census):
    df_census = (
        df_census
        .assign(
            last_name=df_census['last_name_id'].astype(str),
            address=df_census['address_id'].astype(str),
            zipcode=np.nan,
        )
        .drop(columns=['last_name_id', 'address_id'])
    )
    return df_census

df_census_noisy = prep_census_data(df_census)
df_census_noisy

CPU times: user 3.77 s, sys: 752 ms, total: 4.53 s
Wall time: 4.52 s


,simulant_id,age,sex,race_ethnicity,date_of_birth,guardian_1,guardian_2,guardian_1_address_id,guardian_2_address_id,relation_to_household_head,census_year,housing_type,random_seed,year_of_birth,first_name,middle_initial,last_name,address,zipcode
0,3254_0,67.376229,Female,Black,1952-11-15,-1,-1,NaN,NaN,Reference person,2020,Standard,3254,1952,Deanna,S,0.0,6,NaN
1,3254_1,37.501003,Male,Black,1982-10-01,-1,-1,NaN,NaN,Biological child,2020,Standard,3254,1982,Clifton,S,0.0,6,NaN
2,3254_2,31.157344,Male,Black,1989-02-03,-1,-1,NaN,NaN,Biological child,2020,Standard,3254,1989,Christopher,S,0.0,6,NaN
3,3254_3,37.944355,Male,Latino,1982-04-22,-1,-1,NaN,NaN,Reference person,2020,Standard,3254,1982,Travis,J,3.0,7,NaN
4,3254_4,60.986418,Male,White,1959-04-07,-1,-1,NaN,NaN,Reference person,2020,Standard,3254,1959,Davis,T,4.0,8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4943280,3541_278165,0.055643,Female,Latino,2030-03-28,44253,-1,150569.0,NaN,Biological child,2030,Standard,3541,2030,Mila,C,44253.0,150569,NaN
4943281,3541_278166,0.039506,Female,Black,2030-04-03,50097,-1,3.0,NaN,Noninstitutionalized GQ pop,2030,College,3541,2030,Aicha,L,50096.0,3,NaN
4943282,3541_278167,0.052213,Female,Black,2030-03-29,113380,-1,96003.0,NaN,Biological child,2030,Standard,3541,2030,Amila,E,113377.0,96003,NaN
4943283,3541_278168,0.016033,Female,Latino,2030-04-12,140944,-1,179963.0,NaN,Grandchild,2030,Standard,3541,2030,Tallulah,S,140942.0,179963,NaN


In [95]:
df_census_noisy.last_name.map(type).unique()

array([<class 'str'>], dtype=object)

In [113]:
%%time
df_census_noisy = noisify_data.add_noise_to_census(
    df_census_noisy, random_state=88888
)
df_census_noisy

first name
last name
DOB
zip
address
sex
age
race/ethnicity
middle initial
CPU times: user 21.9 s, sys: 572 ms, total: 22.4 s
Wall time: 22.2 s


,simulant_id,age,sex,race_ethnicity,date_of_birth,guardian_1,guardian_2,guardian_1_address_id,guardian_2_address_id,relation_to_household_head,census_year,housing_type,random_seed,year_of_birth,first_name,middle_initial,last_name,address,zipcode
0,3254_0,NaN,Female,Black,1952-11-15,-1,-1,NaN,NaN,Reference person,2020,Standard,3254,1952,Deanna,S,0.0,6,NaN
1,3254_1,37.501003,Male,Black,NaN,-1,-1,NaN,NaN,Biological child,2020,Standard,3254,1982,Clifton,S,0.0,6,NaN
2,3254_2,31.157344,NaN,Black,1989-02-03,-1,-1,NaN,NaN,Biological child,2020,Standard,3254,1989,Christopher,S,0.0,6,NaN
3,3254_3,37.944355,Male,Latino,1982-04-22,-1,-1,NaN,NaN,Reference person,2020,Standard,3254,1982,Travis,J,3.0,7,NaN
4,3254_4,60.986418,Male,White,1959-04-07,-1,-1,NaN,NaN,Reference person,2020,Standard,3254,1959,Davis,T,4.o,8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4943280,3541_278165,0.055643,Female,Latino,2030-03-28,44253,-1,150569.0,NaN,Biological child,2030,Standard,3541,2030,Mila,C,44253.0,150569,NaN
4943281,3541_278166,0.039506,Female,Black,2030-04-03,50097,-1,3.0,NaN,Noninstitutionalized GQ pop,2030,College,3541,2030,Aicha,L,50096.0,3,NaN
4943282,3541_278167,2.052213,Female,Black,2030-03-29,113380,-1,96003.0,NaN,Biological child,2030,Standard,3541,2030,Amila,E,113377.0,96003,NaN
4943283,3541_278168,0.016033,Female,Latino,2030-04-16,140944,-1,179963.0,NaN,Grandchild,2030,Standard,3541,2030,Tallulah,S,140942.0,179963,NaN


## See why `miswrite_zipcode` was failing when zipcodes were all `NaN`

Apparently it can't handle an empty Series, so even though we filtered to non-NaN, we were passing it an empty Series, which made the `.str` accessor complain.

In [97]:
row_prob=0.01
corrupted = df_census_noisy['zipcode'].notna()
corrupted.loc[corrupted] = rng.random(corrupted.sum()) < row_prob
corrupted

0          False
1          False
2          False
3          False
4          False
           ...  
4943280    False
4943281    False
4943282    False
4943283    False
4943284    False
Name: zipcode, Length: 4943285, dtype: bool

In [98]:
corrupted.sum()

0

In [99]:
rng.random(corrupted.sum())

array([], dtype=float64)

In [101]:
# df.loc[corrupted, colname] = noise_function(df.loc[corrupted, colname], *args, **kwargs)
df_census_noisy.loc[corrupted, 'zipcode']

Series([], Name: zipcode, dtype: float64)

In [102]:
corruption.miswrite_zipcode(
    df_census_noisy.loc[corrupted, 'zipcode'],
    .1, .1, .1, rng,
)

AttributeError: Can only use .str accessor with string values!

In [105]:
df_census_noisy.zipcode.astype(str)[0]

'nan'

# Test updated noise on WIC data

In [110]:
df_wic_noisy

,simulant_id,last_name_id,age,sex,race_ethnicity,date_of_birth,address_id,guardian_1,guardian_2,guardian_1_address_id,guardian_2_address_id,wic_year,random_seed,year_of_birth,first_name,middle_initial
0,7359_65,65.0,8.553950,Female,Black,2012-05-21,35,-1,-1,NaN,NaN,2021,7359,2012,NaN,M
1,7359_116,116.0,25.505874,Female,Black,1995-06-08,53,-1,-1,NaN,NaN,2021,7359,1995,NaN,A
2,7359_212,211.0,30.291655,Female,Latino,1990-08-25,88,-1,-1,NaN,NaN,2021,7359,1990,NaN,L
3,7359_248,248.0,23.891573,Female,Black,1997-01-18,97912,-1,-1,NaN,NaN,2021,7359,1997,NaN,M
4,7359_267,266.0,37.072670,Female,White,1983-11-13,109,-1,-1,NaN,NaN,2021,7359,1983,Maria,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593011,5020_277602,231367.0,0.041138,Male,Asian,2030-01-08,152642,231367,-1,152642.0,NaN,2030,5020,2030,Grayson,B
593012,5020_277607,237489.0,0.045699,Male,Latino,2030-01-07,135561,237490,-1,135561.0,NaN,2030,5020,2030,NaN,E
593013,5020_277612,242914.0,0.039537,Female,Asian,2030-01-09,167694,242914,-1,167694.0,NaN,2030,5020,2030,Madison,S
593014,5020_277617,5210.0,0.054807,Male,Latino,2030-01-03,97484,5214,-1,97484.0,NaN,2030,5020,2030,Gavin,J


In [111]:
%%time
def prep_wic_data(df_wic):
    df_wic = (
        df_census
        .assign(
            last_name=df_wic['last_name_id'].astype(str),
            address=df_wic['address_id'].astype(str),
            zipcode=np.nan,
        )
        .rename(columns={'middle_initial': 'middle_name'})
        .drop(columns=['last_name_id', 'address_id'])
    )
    return df_wic

df_wic_noisy = prep_wic_data(df_wic)
df_wic_noisy

CPU times: user 2.57 s, sys: 1.44 s, total: 4.01 s
Wall time: 4.01 s


,simulant_id,age,sex,race_ethnicity,date_of_birth,guardian_1,guardian_2,guardian_1_address_id,guardian_2_address_id,relation_to_household_head,census_year,housing_type,random_seed,year_of_birth,first_name,middle_name,last_name,address,zipcode
0,3254_0,67.376229,Female,Black,1952-11-15,-1,-1,NaN,NaN,Reference person,2020,Standard,3254,1952,Deanna,S,65.0,35,NaN
1,3254_1,37.501003,Male,Black,1982-10-01,-1,-1,NaN,NaN,Biological child,2020,Standard,3254,1982,Clifton,S,116.0,53,NaN
2,3254_2,31.157344,Male,Black,1989-02-03,-1,-1,NaN,NaN,Biological child,2020,Standard,3254,1989,Christopher,S,211.0,88,NaN
3,3254_3,37.944355,Male,Latino,1982-04-22,-1,-1,NaN,NaN,Reference person,2020,Standard,3254,1982,Travis,J,248.0,97912,NaN
4,3254_4,60.986418,Male,White,1959-04-07,-1,-1,NaN,NaN,Reference person,2020,Standard,3254,1959,Davis,T,266.0,109,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4943280,3541_278165,0.055643,Female,Latino,2030-03-28,44253,-1,150569.0,NaN,Biological child,2030,Standard,3541,2030,Mila,C,NaN,NaN,NaN
4943281,3541_278166,0.039506,Female,Black,2030-04-03,50097,-1,3.0,NaN,Noninstitutionalized GQ pop,2030,College,3541,2030,Aicha,L,NaN,NaN,NaN
4943282,3541_278167,0.052213,Female,Black,2030-03-29,113380,-1,96003.0,NaN,Biological child,2030,Standard,3541,2030,Amila,E,NaN,NaN,NaN
4943283,3541_278168,0.016033,Female,Latino,2030-04-12,140944,-1,179963.0,NaN,Grandchild,2030,Standard,3541,2030,Tallulah,S,NaN,NaN,NaN


In [114]:
%%time
df_wic_noisy = noisify_data.add_noise_to_wic(
    df_wic_noisy, random_state=5555555
)
df_wic_noisy

first name
last name
DOB
zip
address
sex
race/ethnicity
middle name
CPU times: user 15.1 s, sys: 487 ms, total: 15.6 s
Wall time: 15.5 s


,simulant_id,age,sex,race_ethnicity,date_of_birth,guardian_1,guardian_2,guardian_1_address_id,guardian_2_address_id,relation_to_household_head,census_year,housing_type,random_seed,year_of_birth,first_name,middle_name,last_name,address,zipcode
0,3254_0,67.376229,Female,Black,1952-11-15,-1,-1,NaN,NaN,Reference person,2020,Standard,3254,1952,Deanna,S,65.0,35,NaN
1,3254_1,37.501003,Male,Black,1982-10-01,-1,-1,NaN,NaN,Biological child,2020,Standard,3254,1982,Clifton,S,116.0,53,NaN
2,3254_2,31.157344,Male,Black,1989-02-03,-1,-1,NaN,NaN,Biological child,2020,Standard,3254,1989,Christopher,S,211.0,88,NaN
3,3254_3,37.944355,Male,Latino,1982-04-22,-1,-1,NaN,NaN,Reference person,2020,Standard,3254,1982,Travis,J,248.0,97912,NaN
4,3254_4,60.986418,Male,White,1959-04-07,-1,-1,NaN,NaN,Reference person,2020,Standard,3254,1959,Davis,T,266.0,109,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4943280,3541_278165,0.055643,Female,Latino,2030-03-28,44253,-1,150569.0,NaN,Biological child,2030,Standard,3541,2030,Mila,C,NaN,NaN,NaN
4943281,3541_278166,0.039506,Female,Black,2030-04-03,50097,-1,3.0,NaN,Noninstitutionalized GQ pop,2030,College,3541,2030,Aicha,L,NaN,NaN,NaN
4943282,3541_278167,0.052213,Female,Black,2030-03-29,113380,-1,96003.0,NaN,Biological child,2030,Standard,3541,2030,Amila,E,NaN,NaN,NaN
4943283,3541_278168,0.016033,Female,Latino,2030-04-12,140944,-1,179963.0,NaN,Grandchild,2030,Standard,3541,2030,Tallulah,S,NaN,NaN,NaN
